# Analysis of the Most Streamed Spotify Songs in 2023



## 1. Business Understanding

This initial phase focuses on understanding the project objectives and requirements from a business perspective, then converting this knowledge into a data mining problem definition and a preliminary plan.

## 2. Data Understanding

This phase involves initial data collection and familiarization, including data cleaning, transformation, and exploration to identify quality issues and insights about the data.

In [1]:
# Importing the necessary libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go

from sklearn import preprocessing
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.cluster import KMeans
from statsmodels.formula.api import ols

import folium as fl
import time

# Setting styles for plots
plt.style.use('ggplot')
sns.set_theme(style="whitegrid")

# Ignore warnings in the output
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Load dataset
file_path = "https://raw.githubusercontent.com/diogo-costa-silva/assets/main/data/spotify-2023.csv"
df = pd.read_csv(file_path, encoding='ISO-8859-1')

# Creating a copy of the dataframe for cleaning
df_cleaned = df.copy()

In [3]:
df

,track_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,...,bpm,key,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%
0,Seven (feat. Latto) (Explicit Ver.),"Latto, Jung Kook",2,2023,7,14,553,147,141381703,43,...,125,B,Major,80,89,83,31,0,8,4
1,LALA,Myke Towers,1,2023,3,23,1474,48,133716286,48,...,92,C#,Major,71,61,74,7,0,10,4
2,vampire,Olivia Rodrigo,1,2023,6,30,1397,113,140003974,94,...,138,F,Major,51,32,53,17,0,31,6
3,Cruel Summer,Taylor Swift,1,2019,8,23,7858,100,800840817,116,...,170,A,Major,55,58,72,11,0,11,15
4,WHERE SHE GOES,Bad Bunny,1,2023,5,18,3133,50,303236322,84,...,144,A,Minor,65,23,80,14,63,11,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
948,My Mind & Me,Selena Gomez,1,2022,11,3,953,0,91473363,61,...,144,A,Major,60,24,39,57,0,8,3
949,Bigger Than The Whole Sky,Taylor Swift,1,2022,10,21,1180,0,121871870,4,...,166,F#,Major,42,7,24,83,1,12,6
950,A Veces (feat. Feid),"Feid, Paulo Londra",2,2022,11,3,573,0,73513683,2,...,92,C#,Major,80,81,67,4,0,8,6
951,En La De Ella,"Feid, Sech, Jhayco",3,2022,10,20,1320,0,133895612,29,...,97,C#,Major,82,67,77,8,0,12,5


The dataframe contains various features related to songs, artists, and their attributes or performance metrics across different platforms.

In order to better understand each feature present in the dataset, here's a quick overview of the dataset columns based on the initial few rows:

- track_name: The title of the tracks.
- artist(s)_name: Names of the artist(s) associated with each track.
- artist_count: The number of artists contributing to each track.
- released_year, released_month, released_day: The release date components for each track.
- Various metrics representing the track's presence and popularity on different music streaming platforms: in_spotify_playlists, in_spotify_charts, streams, in_apple_playlists, in_apple_charts, in_deezer_playlists, in_deezer_charts, in_shazam_charts
- bpm: The tempo of the track, measured in beats per minute.
- key: The key in which the track is composed.
- mode: The mode of the track (major or minor).
- Various metrics representing the track's musical qualities, including danceability_%, valence_%, energy_%, acousticness_%, instrumentalness_%, liveness_%, speechiness_%.



In [4]:
df.shape

(953, 24)

In [5]:
df.columns

Index(['track_name', 'artist(s)_name', 'artist_count', 'released_year',
       'released_month', 'released_day', 'in_spotify_playlists',
       'in_spotify_charts', 'streams', 'in_apple_playlists', 'in_apple_charts',
       'in_deezer_playlists', 'in_deezer_charts', 'in_shazam_charts', 'bpm',
       'key', 'mode', 'danceability_%', 'valence_%', 'energy_%',
       'acousticness_%', 'instrumentalness_%', 'liveness_%', 'speechiness_%'],
      dtype='object')

### Check Data Quality

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 953 entries, 0 to 952
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   track_name            953 non-null    object
 1   artist(s)_name        953 non-null    object
 2   artist_count          953 non-null    int64 
 3   released_year         953 non-null    int64 
 4   released_month        953 non-null    int64 
 5   released_day          953 non-null    int64 
 6   in_spotify_playlists  953 non-null    int64 
 7   in_spotify_charts     953 non-null    int64 
 8   streams               953 non-null    object
 9   in_apple_playlists    953 non-null    int64 
 10  in_apple_charts       953 non-null    int64 
 11  in_deezer_playlists   953 non-null    object
 12  in_deezer_charts      953 non-null    int64 
 13  in_shazam_charts      903 non-null    object
 14  bpm                   953 non-null    int64 
 15  key                   858 non-null    ob

In [29]:
# Checking for missing values
missing_values = df.isnull().sum()
missing_values

track_name               0
artist(s)_name           0
artist_count             0
released_year            0
released_month           0
released_day             0
in_spotify_playlists     0
in_spotify_charts        0
streams                  0
in_apple_playlists       0
in_apple_charts          0
in_deezer_playlists      0
in_deezer_charts         0
in_shazam_charts        50
bpm                      0
key                     95
mode                     0
danceability_%           0
valence_%                0
energy_%                 0
acousticness_%           0
instrumentalness_%       0
liveness_%               0
speechiness_%            0
dtype: int64

In [30]:
# Checking for duplicates
num_duplicates = df.duplicated().sum()
num_duplicates

0

### Explore unique feature values

In [7]:
#df['released_year'].unique()

In [8]:
#df['released_month'].unique()

In [9]:
#df['released_day'].unique()

In [10]:
#df['streams'].unique()

 it seems there's a peculiar value: 'BPM110KeyAModeMajorDanceability53Valence75Energy69Acousticness7Instrumentalness0Liveness17Speechiness3'. This doesn't follow the numerical pattern that we would expect for a column that's supposed to represent streaming counts.

In [11]:
#df['in_deezer_playlists'].unique()

In [12]:
#df['in_shazam_charts'].unique()

In [13]:
#df['bpm'].unique()

In [14]:
#df['mode'].unique()

In [15]:
#df['danceability_%'].unique()

In [16]:
#df['liveness_%'].unique()

In [17]:
#df['instrumentalness_%'].unique()

In [18]:
#df['speechiness_%'].unique()

### 2.1. Data Pre-processing

Before we dive into individual data cleaning tasks, let's summarize the initial steps we need to undertake for <b>Data Cleaning and Transformation</b>:
<br>
1. Transforming Date Features:
<br>
The 'released_year', 'released_month', and 'released_day' fields are currently separate and in integer format. We need to combine these into a single datetime object to allow more efficient temporal analysis.
<br>
2. Cleaning Specific Fields:
<br>
The 'streams' field appears to have an inconsistent entry which we'll need to investigate and clean.
The 'in_deezer_playlists' and 'in_shazam_charts' fields contain numbers with commas, which should be standard integers. We'll convert these.
<br>
3. Reviewing Categorical Variables:
<br>
The 'key' and 'mode' fields are non-numeric and could be considered categorical. We'll review these to decide on the most appropriate treatment, potentially converting them into a category type for efficient processing.
<br>
4. Extended Data Exploration:
<br>
Once the data is cleaned, we will perform an extensive exploratory data analysis (EDA) to uncover insights, patterns, and potential issues in the data. This EDA will involve statistical summaries, visualizations, and various other techniques to understand the data deeply.



1. Convert 'released_year', 'released_month', and 'released_day' into a single datetime object.
2. Clean the 'streams' column and convert its data type.
3. Remove commas from 'in_deezer_playlists' and 'in_shazam_charts' and convert them to integers.
4. Discuss the potential conversion of 'key' and 'mode' into category types.
5. Handle NaN values in ‘in_shazam_charts’ and ‘key’.

We'll start by addressing the first item on our data cleaning list: converting the 'released_year', 'released_month', and 'released_day' columns into a single datetime column. This transformation is important because it allows for more efficient handling of the data, particularly for operations that involve date calculations, filtering, and aggregation.

In [19]:
# Step 1: Ensure the year, month, and day columns are integers (they should already be)
for col in ['released_year', 'released_month', 'released_day']:
    df_cleaned[col] = df_cleaned[col].astype(int)

# Step 2: Combine the year, month, and day into a single column (as a string)
df_cleaned['release_date'] = df_cleaned['released_year'].astype(str) + '-' + \
                             df_cleaned['released_month'].astype(str).str.zfill(2) + '-' + \
                             df_cleaned['released_day'].astype(str).str.zfill(2)  # zfill ensures a format like 2023-07-14

# Step 3: Convert the 'release_date' column to a datetime object
df_cleaned['release_date'] = pd.to_datetime(df_cleaned['release_date'], format='%Y-%m-%d')

# Step 4: (Optional) Drop the original 'released_year', 'released_month', and 'released_day' columns
# We will retain these columns for now, as they might be useful for analysis later on.

# Display the first few rows of the cleaned dataframe to verify our changes
df_cleaned[['released_year', 'released_month', 'released_day', 'release_date']].head()


,released_year,released_month,released_day,release_date
0,2023,7,14,2023-07-14
1,2023,3,23,2023-03-23
2,2023,6,30,2023-06-30
3,2019,8,23,2019-08-23
4,2023,5,18,2023-05-18


The 'released_year', 'released_month', and 'released_day' columns have been successfully combined into a new 'release_date' column, with the date represented as a datetime object. This format is more suitable for any subsequent operations that involve date calculations or aggregations.

Here's a brief overview of what we did:

We confirmed that the 'released_year', 'released_month', and 'released_day' columns were indeed integers.
We concatenated these columns into a single 'release_date' column, ensuring proper zero-padding for single-digit months and days to maintain the 'YYYY-MM-DD' format.
We converted the 'release_date' column from a string to a datetime object using pandas' to_datetime method, providing the specific format to ensure accurate parsing.
We opted to retain the original year, month, and day columns, considering they might be useful for certain types of analysis where these individual components are needed.
With this transformation, operations that require date functionality will be much more straightforward, such as sorting by date, calculating time spans, and aggregating based on specific time periods (e.g., monthly or yearly).

Next, we will handle the 'streams' column, which appears to have an inconsistent entry. We'll need to clean this data and convert the column to the appropriate data type.

Here are the steps we'll take to clean this column:

1. Investigate the unusual entry in the 'streams' column and determine whether it's an isolated case or part of a broader issue.
2. If it's an isolated incident, we may directly correct this entry. If it's a systematic issue, we'll need a more general approach.
3. Convert the 'streams' column to the appropriate numerical type (integer) after handling the anomalous entries.
4. Consider handling any other potential outliers or anomalies that could affect the analysis.

Let's begin by examining the problematic value and understanding the extent of this issue within the column.

In [20]:
# Step 1: Investigate the unusual entry in the 'streams' column.
problematic_entries = df_cleaned[df_cleaned['streams'].astype(str).str.contains(r'[a-zA-Z]')]

# Return the problematic entries for examination
problematic_entries[['track_name', 'streams']]


,track_name,streams
574,Love Grows (Where My Rosemary Goes),BPM110KeyAModeMajorDanceability53Valence75Energy69Acousticness7Instrumentalness0Liveness17Speechiness3


The investigation reveals that there is indeed an anomalous entry within the 'streams' column. Specifically, the track "Love Grows (Where My Rosemary Goes)" has a string of non-numeric characters, which appears to be a concatenation of various song attributes rather than the expected numerical value representing stream counts.

Given that this is an isolated incident, we have a few approaches to rectify this:

- Correction: If we know the correct number of streams or can retrieve it, we could directly replace the incorrect entry.
- Removal: We could remove this specific entry from our dataset to prevent it from skewing any analytical results.
- Imputation: In the absence of the correct data, we might impute a value based on similar tracks or the average number of streams, though this approach can introduce bias.

For the sake of maintaining the integrity of the dataset size and structure, and in the absence of the actual value, we might lean towards imputation. However, the ideal approach would be to retrieve the correct data if accessible.

In this scenario, without the actual streaming count, we'll opt to replace this entry with a 'NaN' so it won't interfere with numerical conversions and calculations. Later, we can decide if we want to impute an estimated value or handle the 'NaN' in another manner.

In [21]:
# Replace the problematic value with NaN (represented as np.nan in the dataset)
df_cleaned.loc[df_cleaned['streams'].astype(str).str.contains(r'[a-zA-Z]'), 'streams'] = np.nan

# Convert the 'streams' column to a float (since we introduced NaN, we can't convert to int directly)
df_cleaned['streams'] = df_cleaned['streams'].astype(float)

# Check if the conversion was successful and how the 'streams' column looks now
df_cleaned['streams'].describe()  # This will give us statistical info including count (to check if we have one less entry)

count    9.520000e+02
mean     5.141374e+08
std      5.668569e+08
min      2.762000e+03
25%      1.416362e+08
50%      2.905309e+08
75%      6.738690e+08
max      3.703895e+09
Name: streams, dtype: float64

The cleanup for the 'streams' column was successful. We replaced the anomalous entry with 'NaN' and converted the rest of the column to float type (since integers can't handle NaN values).

This column is now clean for numerical operations and analyses, although we'll need to decide later how to handle the 'NaN' entry, whether by imputation, removal, or some other strategy.

Next, we'll address the 'in_deezer_playlists' and 'in_shazam_charts' columns, which contain numbers with commas and should be converted to integers.

Here's our plan of action:

1. Replace the commas from the 'in_deezer_playlists' and 'in_shazam_charts' columns to eliminate the thousands separators.
2. Convert these cleaned columns to integers to enable numerical operations.
3. Verify the success of these operations by checking the data types or performing statistical summaries.


In [22]:
# Step 1: Remove commas from the specified columns
df_cleaned['in_deezer_playlists'] = df_cleaned['in_deezer_playlists'].str.replace(',', '').astype(float)
df_cleaned['in_shazam_charts'] = df_cleaned['in_shazam_charts'].str.replace(',', '').astype(float)

# Convert these columns to integers (we use float first because NaN values cannot be converted to int)
# Here, we are not converting to int after replacing commas because of the presence of NaN values.

# Step 3: Verify the operations by checking the data types of these columns and view statistical summaries.
data_types = df_cleaned[['in_deezer_playlists', 'in_shazam_charts']].dtypes

# Display the data types of the cleaned columns
data_types

in_deezer_playlists    float64
in_shazam_charts       float64
dtype: object

In [23]:
#df_cleaned['in_shazam_charts'].unique()

The cleaning for the 'in_deezer_playlists' and 'in_shazam_charts' columns was successful. We removed the commas and converted the values to the float data type. We used float instead of integer due to the presence of 'NaN' values, which are not compatible with the integer type.

These columns are now prepared for numerical analysis and mathematical operations, keeping in mind that we have 'NaN' values that we might need to address later, depending on the specific requirements of our subsequent analysis.

Next, we should discuss the potential categorization of the 'key' and 'mode' columns and then handle the NaN values in the 'in_shazam_charts' and 'key' columns.

The 'key' and 'mode' columns in your dataset represent categorical data, indicating the musical key and mode (major or minor) of each track. While these are represented as strings (object type) in your dataset, converting them to a category data type can be beneficial for several reasons:

- Efficiency: Category data type often uses less memory and can speed up operations like sorting and comparison.
- Integrity: It restricts the data to a specific set of values, ensuring consistency.
- Usefulness for Analysis: Categorical data is handy for statistical methods that are designed to handle categories rather than numerical data, and it's essential for certain visualizations and groupings.

Here's our plan for this part:

1. Examine the unique values in 'key' and 'mode' to understand the range of categories we're dealing with.
2. Convert 'key' and 'mode' to the category data type.
3. Validate the conversion.

In [24]:
# Step 1: Examine the unique values in 'key' and 'mode'
unique_keys = df_cleaned['key'].unique()
unique_modes = df_cleaned['mode'].unique()

# Step 2: Convert 'key' and 'mode' to the category data type
df_cleaned['key'] = df_cleaned['key'].astype('category')
df_cleaned['mode'] = df_cleaned['mode'].astype('category')

# Step 3: Validate the conversion by checking the new data types
new_data_types = df_cleaned[['key', 'mode']].dtypes

unique_keys, unique_modes, new_data_types


(array(['B', 'C#', 'F', 'A', 'D', 'F#', nan, 'G#', 'G', 'E', 'A#', 'D#'],
       dtype=object),
 array(['Major', 'Minor'], dtype=object),
 key     category
 mode    category
 dtype: object)

The conversion was successful. Let's review what we've accomplished:

1. We examined the unique values in both the 'key' and 'mode' columns. The 'key' column contains various musical keys, and there are 'NaN' values present, indicating missing data. The 'mode' column specifies whether the track is in a major or minor mode.

- Unique keys: 'B', 'C#', 'F', 'A', 'D', 'F#', 'G#', 'G', 'E', 'A#', 'D#', and NaN (missing values)
- Unique modes: 'Major', 'Minor'

2. We successfully converted both columns to the category data type, as reflected in the data types' confirmation:

- 'key': category
- 'mode': category


These categorical data types are more efficient for storage and are beneficial for analysis, particularly when dealing with non-numeric data that has a limited set of values. They also make the dataset more consistent and easier to manage, especially for operations that involve grouping, filtering, or creating visuals based on these specific categories.

Next, we need to address the 'NaN' values in the 'in_shazam_charts' and 'key' columns. Handling missing data is crucial as it can significantly impact the results of your analysis.

Handling missing data is a critical step in data cleaning because improper handling can lead to biased results. There are several strategies to deal with missing data, each with its pros and cons:

- Deletion: This is the simplest strategy, involving the removal of rows with missing data. However, it can lead to a loss of information if the dataset isn't large.
- Imputation: This strategy involves filling in the missing values based on other information. Common methods include using the mean, median, mode, or using predictive models. This method can introduce bias if not done carefully.
- Flagging and Filling: This involves two steps: flagging missing values with an indicator and then filling the missing values with a non-null value (zero, mean, etc.). This helps maintain the size of the dataset.
- Leave as Missing: For some algorithms, you might leave them as missing (NaN), especially algorithms that can handle missing values inherently.
We need to consider the nature of our data and the context of our analysis before choosing a strategy. Given that 'key' is categorical and 'in_shazam_charts' is numerical, the approach may differ.

Let's start by understanding the extent of missing data in these columns. We'll calculate the percentage of missing values for each column and then proceed based on the results.

In [32]:
# Calculate the total number of missing values in each column
missing_values = df_cleaned[['in_shazam_charts', 'key']].isnull().sum()

# Calculate the percentage of missing values
total_rows = df_cleaned.shape[0]
percentage_missing = (missing_values / total_rows) * 100

percentage_missing

in_shazam_charts    5.24659
key                 9.96852
dtype: float64

The calculation of missing data reveals the following:

- 'in_shazam_charts': Approximately 5.25% of the entries are missing.
- 'key': Around 9.97% of the entries are missing.
Given these percentages, we need to choose a strategy that balances data integrity with analytical usefulness. Here are the proposed strategies for each column:

1. 'in_shazam_charts' (Numerical): Since the missing percentage is relatively low, we could opt for imputation to retain as much data as possible. Using the median or mean of the existing values could be a suitable approach because these are robust against outliers. Alternatively, if there's a reason to believe that a '0' is a valid substitute (perhaps the song did not make it to the charts), we could use zero imputation.

2. 'key' (Categorical): Deletion is often less desirable for categorical data, especially under 10% missing, as it could lead to biased analyses. A common strategy is to impute with the most frequent category (mode) or to create a separate category, such as 'Unknown' or 'Not Available', to acknowledge the absence of data. This method keeps the dataset's size and can provide additional insights during analysis.

We'll proceed with these strategies. Let's start with the 'in_shazam_charts' column. We'll impute the missing values with the median (to avoid being affected by potential outliers).

In [34]:
# Step 1: Add 'Unknown' to the existing categories of the 'key' column
df_cleaned['key'] = df_cleaned['key'].cat.add_categories('Unknown')

# Step 2: Impute missing values with 'Unknown'
df_cleaned['key'].fillna('Unknown', inplace=True)

# Verify if the missing values are handled
new_missing_values = df_cleaned[['in_shazam_charts', 'key']].isnull().sum()
new_missing_values


in_shazam_charts    0
key                 0
dtype: int64

The missing data handling was successful. We added 'Unknown' as a category to the 'key' column and then filled in the missing entries. Both the 'in_shazam_charts' and 'key' columns now have zero missing values, as evidenced by the counts.

Here's a summary of what we did:

1. 'in_shazam_charts': We calculated the median of the existing values and used this to impute the missing data. This approach ensures that we do not introduce significant bias into the distribution of this column.

2. 'key': We added 'Unknown' as a new category and then filled in the missing data with this value. This strategy acknowledges the absence of data while maintaining the dataset's consistency and size.

With these steps, we've preserved the integrity of our dataset and avoided potential biases associated with dropping rows or imputing them without acknowledgment of their missing nature.

This takes care of the necessary data cleaning and preparation tasks for now. The dataset is now in a much better state for any further analysis you wish to perform.

## Saving the Cleaned Dataset
Let's start by saving your cleaned data (df_cleaned) as a new CSV file. This practice is essential for several reasons:

- Preservation of Clean Data: You ensure that all the cleaning and transformation steps applied do not have to be repeated. It saves time and prevents errors in re-execution.
- Version Control: It helps in maintaining the original version of the dataset intact for any backtracking, comparison, or audit purposes.
- Ease of Sharing: Clean datasets can be easily shared with others working on the project, ensuring everyone uses the same, consistent data.
I will now save df_cleaned as a new CSV file.

In [38]:
# Define the path for the new cleaned data file
cleaned_file_path = '/Users/diogosilva/Desktop/projeto_metyis/spotify_cleaned.csv'

# Save the cleaned DataFrame to a new CSV file
df_cleaned.to_csv(cleaned_file_path, index=False)  # The index=False parameter prevents writing row indices

# Provide the path for confirmation
cleaned_file_path

'/Users/diogosilva/Desktop/projeto_metyis/spotify_cleaned.csv'

### In-depth Data Exploration and Analysis

Now that we have a clean dataset, we can dive into a comprehensive data exploration and analysis phase. This stage is crucial because it allows us to extract meaningful insights, identify patterns and trends, and perhaps discover hidden correlations between different variables. Here's a detailed plan:

1. Descriptive Statistics:
- Purpose: Understand basic characteristics of the data.
- Methods: Calculate mean, median, mode, range, quartiles, and standard deviation for numerical data. For categorical data, analyze the frequency distribution.
- Outcome: Establish a foundational understanding of the data's central tendencies and dispersion.
  
2. Visual Data Exploration:
- Purpose: Identify patterns, spot anomalies, and form hypotheses.
- Methods: Use histograms, box plots, and scatter plots for numerical data. For categorical data, use bar charts and pie charts. Heatmaps can show correlations.
- Outcome: Visual insights into distributions, correlations, and potential outliers or anomalies.

3. Temporal Analysis:
- Purpose: Explore trends over time.
- Methods: Time-series analysis on variables like 'streams' or 'in_spotify_playlists' to identify trends, seasonal patterns, or irregularities.
- Outcome: Insights into how song attributes or popularity metrics have evolved.

4. Feature Relationships and Correlations:
- Purpose: Understand how different variables influence each other.
- Methods: Correlation matrices, scatter plots, and possibly more advanced statistical methods (like chi-square tests for categorical data).
- Outcome: Identification of significant positive or negative relationships between features.

5. Advanced Segment Analysis:
- Purpose: Deep dive into subsets of data.
- Methods: Grouping or segmenting the data by certain criteria (e.g., by 'artist', 'key', or 'mode') and performing a comparative analysis.
- Outcome: Comparative insights, such as how different artists' songs perform or how song characteristics differ between major and minor modes.

6. Anomaly Detection:
- Purpose: Identify unusual data points that deviate from the norm.
- Methods: Statistical methods (like Z-score, IQR) or visual methods (like studying the scatter plots or box plots).
- Outcome: Detection of outliers that could be errors or valuable insights.

7. Hypothesis Testing:
- Purpose: Confirm or reject assumptions.
- Methods: T-tests, ANOVA, or non-parametric tests to confirm the statistical significance of observations.
- Outcome: Validated or refuted hypotheses, providing depth to our insights.

8. Predictive Analytics (Optional):
- Purpose: Predict outcomes based on the data.
- Methods: Regression analysis, or machine learning algorithms like random forest or neural networks, if the project scope allows.
- Outcome: Models that can predict, for example, a song's popularity based on its attributes.

9. Report and Communicate Findings:
- Purpose: Share insights and recommendations.
- Methods: Compile the analyses into a comprehensive report or presentation, using clear visualizations and concise interpretations.
- Outcome: A ready-to-share report that conveys the story behind the data, supporting decision-making.

## Exploratory Data Analysis (EDA) 

This stage is crucial as it involves understanding the distribution and characteristics of the data, which informs the subsequent steps of feature engineering and model selection.

We'll begin with univariate analysis, which involves analyzing one variable at a time, and then move to multivariate analysis, where we examine relationships between multiple variables. Throughout this process, we'll create visualizations and use statistical measures to understand the data better.

### Univariate analysis

This process involves examining one variable at a time and is foundational in understanding the data's overall characteristics. We'll use both graphical and statistical methods for a comprehensive analysis.

Here's how we'll proceed:

1. Selection of Variables: We'll select a few key variables to start. Given the nature of your data, it would be insightful to begin with variables that represent the characteristics of the songs, such as 'danceability_%', 'energy_%', and 'valence_%'. These features often have interesting distributions and can tell us a lot about the dataset's nature.

2. Statistical Summary: For each variable, we'll calculate descriptive statistics, including measures of central tendency (mean, median) and measures of dispersion (range, variance, standard deviation, skewness, kurtosis). This gives us a clear, concise view of the data's central values and variability.

3. Visualization: We'll create visual representations of each variable using histograms or box plots. Histograms will allow us to see the data distribution and identify any skewness. In contrast, box plots provide insights into the data's quartiles and potential outliers.

4. Interpretation: For each analysis, we'll provide an interpretation of the statistics and visualizations, explaining what they tell us about the data. This step is crucial for understanding any implications or interesting characteristics of the variables.

By conducting a thorough univariate analysis, we lay a solid foundation for the more complex bivariate and multivariate analyses to follow, ensuring we fully understand the variables before we start exploring their interactions.



### Basic descriptive statistics and general data checks

We'll check for any missing or duplicate values and understand the data types and summary statistics of each column. This step is crucial for deciding how to handle preprocessing in the Data Preparation phase.

In [25]:
# Descriptive statistics for numerical columns
desc_stats = df.describe()
desc_stats

,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,in_apple_playlists,in_apple_charts,in_deezer_charts,bpm,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%
count,953.000000,953.000000,953.000000,953.000000,953.000000,953.000000,953.000000,953.000000,953.000000,953.000000,953.00000,953.000000,953.000000,953.000000,953.000000,953.000000,953.000000
mean,1.556139,2018.238195,6.033578,13.930745,5200.124869,12.009444,67.812172,51.908709,2.666317,122.540399,66.96957,51.431270,64.279119,27.057712,1.581322,18.213012,10.131165
std,0.893044,11.116218,3.566435,9.201949,7897.608990,19.575992,86.441493,50.630241,6.035599,28.057802,14.63061,23.480632,16.550526,25.996077,8.409800,13.711223,9.912888
min,1.000000,1930.000000,1.000000,1.000000,31.000000,0.000000,0.000000,0.000000,0.000000,65.000000,23.00000,4.000000,9.000000,0.000000,0.000000,3.000000,2.000000
25%,1.000000,2020.000000,3.000000,6.000000,875.000000,0.000000,13.000000,7.000000,0.000000,100.000000,57.00000,32.000000,53.000000,6.000000,0.000000,10.000000,4.000000
50%,1.000000,2022.000000,6.000000,13.000000,2224.000000,3.000000,34.000000,38.000000,0.000000,121.000000,69.00000,51.000000,66.000000,18.000000,0.000000,12.000000,6.000000
75%,2.000000,2022.000000,9.000000,22.000000,5542.000000,16.000000,88.000000,87.000000,2.000000,140.000000,78.00000,70.000000,77.000000,43.000000,0.000000,24.000000,11.000000
max,8.000000,2023.000000,12.000000,31.000000,52898.000000,147.000000,672.000000,275.000000,58.000000,206.000000,96.00000,97.000000,97.000000,97.000000,91.000000,97.000000,64.000000


## 3. Data Preparation

This stage often consumes the most amount of time in data science projects. It covers all activities needed to construct the final dataset from the initial raw data, including cleaning, feature selection, data transformation, and scaling.

## 4. Modeling

Various modeling techniques are selected and applied, and their parameters are calibrated to optimal values, usually through iteration and cross-validation.


## 5. Evaluation

After one or more models are developed, they need to be evaluated with respect to the business objectives. This phase helps determine the best model that meets the business objectives, possibly leading to a decision to deploy the model.


## 6. Deployment

The knowledge gained will need to be organized and presented in a way that the customer can use it. It involves deploying the chosen model into a real-world scenario for decision-making.